## 0. Import Libraries

In [4]:
from datetime import datetime, timedelta
from glob import glob

import numpy as np
import pandas as pd
import pyproj
import rioxarray
import salem
import xarray as xr
from shapely.geometry import mapping

import cartopy
import matplotlib.pyplot as plt
import proplot
from matplotlib.colors import BoundaryNorm, ListedColormap
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader
from dea_tools.spatial import xr_vectorize, xr_rasterize
from wrf import ll_to_xy
import matplotlib.patheffects as pe
from netCDF4 import Dataset
from SPAEF_metric import SPAEF

import warnings
warnings.filterwarnings('ignore')

## 1. Simulation Data

In [5]:
variable = 'PRCP'
general_path = 'data'

# ensemble members
micro_options = [
    'LIN',
    'THOMPSON',
    'WSM6'
]

luse_options = [
    'urban',
    'nourban'
]

### 1.1 ERA5

#### 1.1.1 Data Options

In [6]:
run_data = 'era5'

# data path
data_path = glob(fr'{general_path}/*{run_data}**{variable}*')

In [7]:
data_path

['data/era5_LIN_nourban_PRCP_wrfout_d03_2017-07-18.nc',
 'data/era5_LIN_urban_PRCP_wrfout_d03_2017-07-18.nc',
 'data/era5_THOMPSON_nourban_PRCP_wrfout_d03_2017-07-18.nc',
 'data/era5_THOMPSON_urban_PRCP_wrfout_d03_2017-07-18.nc',
 'data/era5_WSM6_nourban_PRCP_wrfout_d03_2017-07-18.nc',
 'data/era5_WSM6_urban_PRCP_wrfout_d03_2017-07-18.nc']

#### 1.1.2 Open Data

In [8]:
# open data
dt = xr.open_mfdataset(data_path,
                       concat_dim='ens')[variable]

# instead of 201 grid we want 200 grid in each direction
dt = dt.isel(south_north=slice(0,200),
             west_east=slice(0,200))

# assign projection and dim info
dt = dt.rio.write_crs(dt.attrs['pyproj_srs'])
dt_era5 = dt.rio.set_spatial_dims(x_dim='west_east',
                             y_dim='south_north')

# sum over time dimension
dt_era5_sum = dt_era5.sum(dim='time')

In [9]:
dt_era5_sum

<xarray.DataArray 'PRCP' (ens: 6, south_north: 200, west_east: 200)>
dask.array<sum-aggregate, shape=(6, 200, 200), dtype=float32, chunksize=(1, 200, 200), chunktype=numpy.ndarray>
Coordinates:
    lat          (south_north, west_east) float32 dask.array<chunksize=(200, 200), meta=np.ndarray>
    lon          (south_north, west_east) float32 dask.array<chunksize=(200, 200), meta=np.ndarray>
  * west_east    (west_east) float64 -1.09e+05 -1.08e+05 ... 8.9e+04 9e+04
  * south_north  (south_north) float64 -1.09e+05 -1.08e+05 ... 8.9e+04 9e+04
  * ens          (ens) object 'LIN_nourban_PRCP' ... 'WSM6_urban_PRCP'
    spatial_ref  int64 0

#### 1.1.3 Define Members

In [10]:
# ensemble member list
ensemble_members = list(dt.ens.values)

# urban and nourban members
urban_members = [ens for ens in ensemble_members if not 'nourban' in ens ]
nourban_members = [ens for ens in ensemble_members if 'nourban' in ens ]

#### 1.1.4 Ensemble Mean For Urban and Nourban

In [11]:
dt_era5_sum_urban_ens = dt_era5_sum.sel(ens=urban_members).mean(dim='ens')
dt_era5_urban_ens = dt_era5.sel(ens=urban_members).mean(dim='ens')

In [12]:
dt_era5_sum_urban_ens

<xarray.DataArray 'PRCP' (south_north: 200, west_east: 200)>
dask.array<mean_agg-aggregate, shape=(200, 200), dtype=float32, chunksize=(200, 200), chunktype=numpy.ndarray>
Coordinates:
    lat          (south_north, west_east) float32 dask.array<chunksize=(200, 200), meta=np.ndarray>
    lon          (south_north, west_east) float32 dask.array<chunksize=(200, 200), meta=np.ndarray>
  * west_east    (west_east) float64 -1.09e+05 -1.08e+05 ... 8.9e+04 9e+04
  * south_north  (south_north) float64 -1.09e+05 -1.08e+05 ... 8.9e+04 9e+04
    spatial_ref  int64 0

### 1.2 GFS

#### 1.2.1 Data Options

In [13]:
run_data = 'gfs' # gfs in normal

# data path
data_path = glob(fr'{general_path}/*{run_data}**{variable}*')

In [14]:
data_path

['data/gfs_LIN_nourban_PRCP_wrfout_d03_2017-07-18.nc',
 'data/gfs_LIN_urban_PRCP_wrfout_d03_2017-07-18.nc',
 'data/gfs_THOMPSON_nourban_PRCP_wrfout_d03_2017-07-18.nc',
 'data/gfs_THOMPSON_urban_PRCP_wrfout_d03_2017-07-18.nc',
 'data/gfs_WSM6_nourban_PRCP_wrfout_d03_2017-07-18.nc',
 'data/gfs_WSM6_urban_PRCP_wrfout_d03_2017-07-18.nc']

#### 1.1.2 Open Data

In [15]:
# open data
dt = xr.open_mfdataset(data_path,
                       concat_dim='ens')[variable]

# instead of 201 grid we want 200 grid in each direction
dt = dt.isel(south_north=slice(0,200),
             west_east=slice(0,200))

# assign projection and dim info
dt = dt.rio.write_crs(dt.attrs['pyproj_srs'])
dt_gfs = dt.rio.set_spatial_dims(x_dim='west_east',
                             y_dim='south_north')

# sum over time dimension
dt_gfs_sum = dt_gfs.sum(dim='time')

In [16]:
dt_gfs_sum

<xarray.DataArray 'PRCP' (ens: 6, south_north: 200, west_east: 200)>
dask.array<sum-aggregate, shape=(6, 200, 200), dtype=float32, chunksize=(1, 200, 200), chunktype=numpy.ndarray>
Coordinates:
    lat          (south_north, west_east) float32 dask.array<chunksize=(200, 200), meta=np.ndarray>
    lon          (south_north, west_east) float32 dask.array<chunksize=(200, 200), meta=np.ndarray>
  * west_east    (west_east) float64 -1.09e+05 -1.08e+05 ... 8.9e+04 9e+04
  * south_north  (south_north) float64 -1.09e+05 -1.08e+05 ... 8.9e+04 9e+04
  * ens          (ens) object 'LIN_nourban_PRCP' ... 'WSM6_urban_PRCP'
    spatial_ref  int64 0

#### 1.1.3 Ensemble Mean For Urban and Nourban

In [17]:
dt_gfs_sum_urban_ens = dt_gfs_sum.sel(ens=urban_members).mean(dim='ens')
dt_gfs_urban_ens = dt_gfs.sel(ens=urban_members).mean(dim='ens')

In [18]:
dt_gfs_sum_urban_ens

<xarray.DataArray 'PRCP' (south_north: 200, west_east: 200)>
dask.array<mean_agg-aggregate, shape=(200, 200), dtype=float32, chunksize=(200, 200), chunktype=numpy.ndarray>
Coordinates:
    lat          (south_north, west_east) float32 dask.array<chunksize=(200, 200), meta=np.ndarray>
    lon          (south_north, west_east) float32 dask.array<chunksize=(200, 200), meta=np.ndarray>
  * west_east    (west_east) float64 -1.09e+05 -1.08e+05 ... 8.9e+04 9e+04
  * south_north  (south_north) float64 -1.09e+05 -1.08e+05 ... 8.9e+04 9e+04
    spatial_ref  int64 0

## 2. Intra-Simulation Metric Calculation

### 2.1 Concat All Ensemble Members

In [49]:
era5_gfs_concat = np.concatenate([dt_era5_sum.sel(ens=urban_members).values, 
               dt_gfs_sum.sel(ens=urban_members).values])

### 2.2 Concat Ensemble Mean

In [50]:
ens_era5_gfs_concat = np.concatenate([dt_era5_sum_urban_ens.values[np.newaxis],
                    dt_gfs_sum_urban_ens.values[np.newaxis]])

### 2.3 Concat All Simulations

In [51]:
simulations = np.concatenate([
    era5_gfs_concat,
    ens_era5_gfs_concat
]) 

In [52]:
np.shape(simulations)

(8, 200, 200)

### 2.4 Calculate Metrics

In [53]:
# data used in simulation numpy array in order
simulation_data_order = [
    'era5-lin',
    'era5-thompson',
    'era5-wsm6',
    'gfs-lin',
    'gfs-thompson',
    'gfs-wsm6',
    'ens-era5',
    'ens-gfs'
]

In [54]:
metrics = np.zeros((8,8), dtype=np.float64)

for i, s1 in enumerate(simulations):
    
    print(fr'next first data is {simulation_data_order[i]}')
    for j, s2 in enumerate(simulations):
        
        print(fr'>> next second data is {simulation_data_order[j]}')
        
        # spaef calculation
        SPAef, alpha, beta, gamma = SPAEF(s1, s2)
        
        metrics[i, j] = SPAef
        
    print('END ------------------')

next first data is era5-lin
>> next second data is era5-lin
>> next second data is era5-thompson
>> next second data is era5-wsm6
>> next second data is gfs-lin
>> next second data is gfs-thompson
>> next second data is gfs-wsm6
>> next second data is ens-era5
>> next second data is ens-gfs
END ------------------
next first data is era5-thompson
>> next second data is era5-lin
>> next second data is era5-thompson
>> next second data is era5-wsm6
>> next second data is gfs-lin
>> next second data is gfs-thompson
>> next second data is gfs-wsm6
>> next second data is ens-era5
>> next second data is ens-gfs
END ------------------
next first data is era5-wsm6
>> next second data is era5-lin
>> next second data is era5-thompson
>> next second data is era5-wsm6
>> next second data is gfs-lin
>> next second data is gfs-thompson
>> next second data is gfs-wsm6
>> next second data is ens-era5
>> next second data is ens-gfs
END ------------------
next first data is gfs-lin
>> next second data is

In [59]:
df = pd.DataFrame(metrics, columns = [simulation_data_order],
             index = simulation_data_order)

In [60]:
df.style.background_gradient(cmap="Reds")